In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-14 22:47:27--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-14 22:47:27 (26.2 MB/s) - ‘minsearch.py’ saved [4073/4073]



In [4]:
import minsearch
!pip install minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import json
import requests

In [4]:
data_url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents-llm.json"

response = requests.get(data_url)
data = response.json()

In [5]:
data

[{'course': 'llm-zoomcamp',
  'documents': [{'text': 'Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.',
    'section': 'General course-related questions',
    'question': 'I just discovered the course. Can I still join?'},
   {'text': "You don't need it. You're accepted. You can also just start learning and submitting homework (while the form is Open) without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
    'section': 'General course-related questions',
    'question': 'Course - I have registered for the [insert-zoomcamp-name]. When can I expect to receive the confirmation email?'},
   {'text': 'The zoom link is only published to instructors/presenters/TAs.\nStudents participate via Youtube Live and submit questions to Slido (link would be pinned in the chat when Alexey goes Live). The video URL should be posted in the announcements chan

In [5]:
documents = []
for course_dict in data:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents

[{'text': 'Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.',
  'section': 'General course-related questions',
  'question': 'I just discovered the course. Can I still join?',
  'course': 'llm-zoomcamp'},
 {'text': "You don't need it. You're accepted. You can also just start learning and submitting homework (while the form is Open) without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
  'section': 'General course-related questions',
  'question': 'Course - I have registered for the [insert-zoomcamp-name]. When can I expect to receive the confirmation email?',
  'course': 'llm-zoomcamp'},
 {'text': 'The zoom link is only published to instructors/presenters/TAs.\nStudents participate via Youtube Live and submit questions to Slido (link would be pinned in the chat when Alexey goes Live). The video URL should be posted in the announcements cha

In [7]:
from minsearch import Index
index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

/workspaces/llm-zoomcamp/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [8]:
index.fit(documents)

In [9]:
from openai import OpenAI
import os
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [10]:
boost = {'question':3.0, 'section':0.5}
results = index.search(
    query='When does the course start?',
    #If there is more than one course : filter_dict={'course':'data-engineering-zoomcamp'}, 
    boost_dict=boost,
    num_results=1
)
    

In [11]:
print(results)

[{'text': 'Summer 2025 (via Alexey).', 'section': 'General course-related questions', 'question': 'When will the course be offered next?', 'course': 'llm-zoomcamp'}]


In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':'When does the course start?'}]
)

In [13]:
response.choices[0].message.content

'Could you please specify which course you are referring to?'

In [14]:
prompt_template = """ 
You are a course instructor. Answer the QUESTION based on the CONTEXT
Use only the facts from the CONTEXT when answering the QUESTION.
If the answer is not in the CONTEXT then output NONE

QUESTION:{question}

CONTEXT:{context}
""".strip()

In [15]:
context = ""
for doc in documents:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}]\n\n"

In [16]:
print(context.strip())

section: General course-related questions
question: I just discovered the course. Can I still join?
answer:Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.]

section: General course-related questions
question: Course - I have registered for the [insert-zoomcamp-name]. When can I expect to receive the confirmation email?
answer:You don't need it. You're accepted. You can also just start learning and submitting homework (while the form is Open) without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.]

section: General course-related questions
question: What is the video/zoom link to the stream for the “Office Hours” or live/workshop sessions?
answer:The zoom link is only published to instructors/presenters/TAs.
Students participate via Youtube Live and submit questions to Slido (link would be pinned in the chat when Alexey goes Live). The video 

In [17]:
prompt = prompt_template.format(question='Will I get a certificate??',
                       context=context).strip()

In [18]:
print(prompt)

You are a course instructor. Answer the QUESTION based on the CONTEXT
Use only the facts from the CONTEXT when answering the QUESTION.
If the answer is not in the CONTEXT then output NONE

QUESTION:Will I get a certificate??

CONTEXT:section: General course-related questions
question: I just discovered the course. Can I still join?
answer:Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.]

section: General course-related questions
question: Course - I have registered for the [insert-zoomcamp-name]. When can I expect to receive the confirmation email?
answer:You don't need it. You're accepted. You can also just start learning and submitting homework (while the form is Open) without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.]

section: General course-related questions
question: What is the video/zoom link to the stream for the “Office Hours”

In [19]:
#Using the content in chatgpt
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':prompt}]
)

In [20]:
response.choices[0].message.content

'Yes, you will get a certificate, but only if you submit your project while submissions are still being accepted. Additionally, you can only receive a certificate if you finish the course with a "live" cohort; self-paced mode does not qualify. It is also necessary to pass the Capstone project and participate in the peer-review of three capstone projects after submitting your own project.'

In [21]:
def search(query):
    boost = {'question':3.0, 'section':0.5}
    result = index.search(
    query=query,
    #If there is more than one course : filter_dict={'course':'data-engineering-zoomcamp'}, 
    boost_dict=boost,
    num_results=20)
    return result

def prompt(result, query):
    prompt_template = """ 
    You are a course instructor. Answer the QUESTION based on the CONTEXT
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the answer is not in the CONTEXT then output NONE
    
    QUESTION:{question}
    
    CONTEXT:{context}
    """.strip()
    context = ""
    for doc in result:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}]\n\n"
    prompt = prompt_template.format(question='Will I get a certificate??',
                       context=context).strip()
    return prompt

def run_llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':prompt}]
    )
    answer = response.choices[0].message.content
    return answer

def rag(query):
    result = search(query)
    gen_prompt = prompt(result, query)
    answer = run_llm(gen_prompt)
    return answer

In [22]:
rag('Will I get a certificate?')

'No, you can only get a certificate if you finish the course with a “live” cohort. Certificates are not awarded for the self-paced mode.'

In [23]:
#To run elastic search
from elasticsearch import Elasticsearch
import elasticsearch
print(f"Elasticsearch client version: {elasticsearch.__version__}")

Elasticsearch client version: (8, 4, 3)


In [26]:
es_client = Elasticsearch("http://localhost:9200")
index_settings ={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
if es_client.indices.exists(index="course-questions"):
    es_client.indices.delete(index="course-questions")
index_name = "course-questions"
es_client.indices.create(
    index=index_name,
    body=index_settings
)

/tmp/ipykernel_136559/3991661907.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [27]:
documents[0]

{'text': 'Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.',
 'section': 'General course-related questions',
 'question': 'I just discovered the course. Can I still join?',
 'course': 'llm-zoomcamp'}

In [28]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 180.75it/s]


In [30]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": 'Will I get a certificate?',
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "llm-zoomcamp"
                }
            }
        }
    }
}

In [32]:
response = es_client.search(
    index=index_name,
    body=search_query
)

/tmp/ipykernel_136559/1039276247.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(


In [36]:
result_doc = []
for hit in response['hits']['hits']:
    result_doc.append(hit['_source'])

In [37]:
result_doc

[{'text': "No, you can only get a certificate if you finish the course with a “live” cohort.\nWe don't award certificates for the self-paced mode. The reason is you need to peer-review 3 capstone(s) after submitting your own project.\nYou can only peer-review projects at the time the course is running; after the form is closed and the peer-review list is compiled.",
  'section': 'General course-related questions',
  'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?',
  'course': 'llm-zoomcamp'},
 {'text': 'Yes, you need to pass the Capstone project to get the certificate. Homework is not mandatory, though it is recommended for reinforcing concepts, and the points awarded count towards your rank on the leaderboard.',
  'section': 'General course-related questions',
  'question': 'I missed the first homework - can I still get a certificate?',
  'course': 'llm-zoomcamp'},
 {'text': 'Issue: I get the notice that due to traffic, I’m on a waitlist

In [39]:
def elastic_search(query):
    search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "llm-zoomcamp"
                }
            }
        }
    }
    }
    response = es_client.search(
    index=index_name,
    body=search_query
    )
    result_doc = []
    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])
    return result_doc

In [40]:
def rag(query):
    result = elastic_search(query)
    gen_prompt = prompt(result, query)
    answer = run_llm(gen_prompt)
    return answer

In [41]:
rag('Will I get a certificate')

/tmp/ipykernel_136559/3664667401.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(


'No, you can only get a certificate if you finish the course with a “live” cohort. Certificates are not awarded for the self-paced mode.'